In [1]:
%%capture 
!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install gdal
!pip install numpy
!pip install rasterio

In [2]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

In [3]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/jameslab/doriana 

dirname = '/content/drive/MyDrive/jameslab/doriana/'

Mounted at /content/drive
/content/drive/MyDrive/jameslab/doriana


In [ ]:
#code from: https://stackoverflow.com/questions/69938501/clipping-raster-through-a-shapefile-using-python


import fiona
import rasterio
import rasterio.mask


src_raster_path = r'CanLaD_20151984_latest_type.tif'

shp_file_path = "BC_shp_proj.shp"

output_raster_path = src_raster_path[:-3] + '_BCmask.tif'

with fiona.open(shp_file_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(src_raster_path) as src:

    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})


with rasterio.open(output_raster_path, "w", **out_meta) as dest:
    dest.write(out_image)

In [6]:
#code from: https://stackoverflow.com/questions/69938501/clipping-raster-through-a-shapefile-using-python


import fiona
import rasterio
import rasterio.mask


src_raster_path = r'CanLaD_20151984_latest_type_BCmask.tif'

from shapely.geometry import Polygon, MultiPolygon, shape, Point
import geopandas as gp
def convert_3D_2D(geometry):
    '''
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    '''
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == 'Polygon':
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == 'MultiPolygon':
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
    return new_geo

geodf_2d = gp.GeoDataFrame.from_file("fires_selected_correct.shp") # plug_in your shapefile
geodf_2d.geometry = convert_3D_2D(geodf_2d.geometry)
shp_file_path = "fires_selected_correct.shp"

output_raster_path = src_raster_path[:-4] + '_FIREmask.tif'


shapes = list(geodf_2d['geometry'])
print(shapes)
with rasterio.open(src_raster_path) as src:

    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})


with rasterio.open(output_raster_path, "w", **out_meta) as dest:
    dest.write(out_image)

[<shapely.geometry.polygon.Polygon object at 0x7f4c810ad210>, <shapely.geometry.polygon.Polygon object at 0x7f4c87f3a610>, <shapely.geometry.multipolygon.MultiPolygon object at 0x7f4c87f24710>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50c10>, <shapely.geometry.multipolygon.MultiPolygon object at 0x7f4c80b50fd0>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b502d0>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50750>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b506d0>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50d10>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50450>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50610>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50c50>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50290>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50b50>, <shapely.geometry.polygon.Polygon object at 0x7f4c80b50a10>, <shapely.geometry.multipolygon.MultiPolygon object at 0x7f4c80b5

In [21]:
#code from: https://gis.stackexchange.com/questions/187877/how-to-polygonize-raster-to-shapely-polygons
import rasterio
from rasterio.features import shapes
mask = None
with rasterio.Env():
    with rasterio.open('CanLaD_20151984_latest_type_BCmask_FIREmask.tif') as src:
        image = src.read(1, masked=True) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)) if v == 2)
        
from shapely.geometry import shape
#results = {k:v for (k,v) in list(results).items()}
print(results)
geoms = list(results)
print(geoms)
print(shape(geoms[0]['geometry']))
df = gpd.GeoDataFrame.from_features(geoms)
print(df)
#df = df[df['raster_val'] == 2]
geom_list = [] 

for val in list(df['geometry']): 
    geom_list.append(val)

new_df = gpd.GeoDataFrame(geometry = geom_list).dissolve()
new_df.to_file('vectorized_dtype3.geojson',driver="GeoJSON")

<generator object <genexpr> at 0x7fade3c3d050>
[{'properties': {'raster_val': 2.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[(-1885770.0, 8341020.0), (-1885770.0, 8340990.0), (-1885740.0, 8340990.0), (-1885740.0, 8341020.0), (-1885770.0, 8341020.0)]]}}, {'properties': {'raster_val': 2.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[(-1888680.0, 8338350.0), (-1888680.0, 8338320.0), (-1888650.0, 8338320.0), (-1888650.0, 8338350.0), (-1888680.0, 8338350.0)]]}}, {'properties': {'raster_val': 2.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[(-1890090.0, 8338200.0), (-1890090.0, 8338170.0), (-1890060.0, 8338170.0), (-1890060.0, 8338200.0), (-1890090.0, 8338200.0)]]}}, {'properties': {'raster_val': 2.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[(-1887030.0, 8335650.0), (-1887030.0, 8335620.0), (-1887000.0, 8335620.0), (-1887000.0, 8335650.0), (-1887030.0, 8335650.0)]]}}, {'properties': {'raster_val': 2.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[(-1887060.0